In [1]:
from datetime import datetime, timedelta
from json import loads as json_loads
from numpy import nan
from pandas import read_parquet
from tqdm.auto import tqdm

Testing Connection to AWS S3

In [2]:
!aws s3 ls s3://

2020-04-14 09:44:35 adc-ada1
2021-10-22 16:25:41 adc-aws-billing
2021-02-20 17:32:47 adc-cloudtrail-logs
2021-06-25 09:56:11 adc-cloudwatch-logs
2022-04-07 10:31:18 adc-de-dev
2021-06-04 09:47:20 adc-ds
2020-11-04 14:08:07 adc-ds-aspirasi-user
2020-11-04 14:09:42 adc-ds-athena-query-result
2021-10-04 11:45:34 adc-ds-cloudtrails
2020-10-20 12:17:11 adc-ds-custom-logs
2020-11-04 13:58:05 adc-ds-data
2021-03-23 23:05:51 adc-ds-detool
2020-11-04 14:11:21 adc-ds-dev
2021-06-15 09:44:15 adc-ds-factdata
2021-10-04 07:14:28 adc-ds-lms
2020-11-04 14:12:29 adc-ds-logging
2021-03-26 16:00:30 adc-ds-ml
2022-03-29 22:22:42 adc-ds-replicated-data
2020-05-19 08:27:10 adc-ds-s3-logs-ap-southeast-1
2021-12-18 21:57:35 adc-ds-scripts
2020-11-04 14:13:02 adc-ds-sftp
2020-03-03 12:26:31 adc-id-reporting
2020-11-12 00:37:47 adc-my-reporting
2021-06-01 21:02:43 adc-production-backups
2021-02-20 12:15:19 adc-production-firewal-log
2021-12-18 21:50:09 adc-replication-example
2021-06-07 21:37:06 adc-vpc-flow-l

# Data URIs from S3 Source

In [3]:
data_cutoff_date = (datetime.today() + timedelta(days=-1)).date()
hour_code = '08' # Get this when checking with aws s3.
data_cutoff_text = data_cutoff_date.strftime('%Y%m%d') + hour_code
dat = {
    'app': {'uri': f's3://adc-ds-lms/aspirasi/prod/lms/adc_lms/{data_cutoff_text}/application_tab/'},
    'contract': {'uri': f's3://adc-ds-lms/aspirasi/prod/lms/adc_lms/{data_cutoff_text}/contracts_tab/'},
    'product': {'uri': f's3://adc-ds-lms/aspirasi/prod/lms/adc_lms/{data_cutoff_text}/product_tab/'},
    'provision': {'uri': f's3://adc-ds-lms/aspirasi/prod/lms/adc_lms/{data_cutoff_text}/provisioning_metadata/'},
    'dot': {'uri': f's3://adc-ds-lms/aspirasi/prod/journey/prejourney_funnel/{data_cutoff_text}/digital_onboarding_tab.parquet/'},
    'repayment': {'uri': f's3://adc-my-reporting/lending/lmsv2_my_mwc_repayment/report_year={data_cutoff_date.year}/report_month={data_cutoff_date.month}/report_day={data_cutoff_date.day}/'},
    'meta': {'uri': f's3://adc-ds-lms/aspirasi/prod/journey/prejourney_funnel/{data_cutoff_text}/metadata_storage.parquet/'}
}

# Load Data from Parquet Referencing URIs

In [4]:
# Load Data
for _tab in tqdm(
    ('dot', 'contract', 'provision', 'meta', 'app'),
    desc='Reading Data',
    unit='table',
    colour='orange'
):
    dat.get(_tab, {})['df'] = read_parquet(dat.get(_tab, {}).get('uri', ''))
    if _tab=='meta':
        dat[_tab]['df']['ms_request_id'] = dat[_tab]['df']['ms_metadata']\
                                               .apply(lambda _: json_loads(_).get('requestID',
                                                                                  nan))
    elif _tab=='provision':
        dat[_tab]['df'].drop(
            index=dat[_tab]['df'].index[dat[_tab]['df']['pm_merchant_id'].eq(0)], 
            inplace=True
        )

Reading Data:   0%|          | 0/5 [00:00<?, ?table/s]

# Unify Dataset Tables

In [5]:
onboard_dat = dat.setdefault('onboarding', {})['df'] = dat['dot']['df'].merge(
    right=dat['meta']['df'],
    how='inner',
    left_on='dot_session_key',
    right_on='ms_tracker_id',
    copy=False
).merge(
    right=dat['provision']['df'],
    how='inner',
    left_on='ms_request_id',
    right_on='pm_originating_request_id',
    copy=False
).merge(
    right=dat['app']['df'],
    how='inner',
    left_on='pm_merchant_id',
    right_on='at_merchant_id',
    copy=False
).merge(
    right=dat['contract']['df'],
    how='inner',
    left_on='at_id',
    right_on='ct_originating_application_id',
    copy=False
)
onboard_dat.replace(to_replace={None: nan}, inplace=True)
onboard_dat = onboard_dat.convert_dtypes()

# Drop Redundant Columns
onboard_dat.drop(columns=['dot_session_key', 'ms_request_id', 'pm_merchant_id', 'at_id'],
                 inplace=True)

# Identify Records by ID
onboard_dat.set_index(keys=['dot_id',
                            'ms_tracker_id',
                            'pm_originating_request_id',
                            'at_merchant_id',
                            'ct_originating_application_id'],
                      drop=True,
                      inplace=True)

## Preview

In [6]:
from math import ceil
def get_sample_size(
    population_size: int,
    confidence_level: float=0.95,
    margin_of_error: float=0.05
) -> int:
    num_num = 0.25 * confidence_level**2
    num_den = margin_of_error**2
    num = num_num * num_den**(-1)
    return ceil(num * (1 + num * population_size**(-1))**(-1))

In [7]:
onboard_dat.sample(n=get_sample_size(onboard_dat.shape[0], 0.9, 0.1), 
                   random_state=42).transpose(copy=False)

dot_id,250169,143883,23713,31674,86617,28319,10505,65848,183481,253898,...,149025,14524,105139,82569,266905,2392,23065,56231,136114,174618
ms_tracker_id,79a840e8-f780-11eb-ae4a-0a5843143288,e578eeff-5159-11eb-a321-0a5843143288,11d23e13-b47e-11ea-a321-0a5843143288,eca1c630-ba00-11ea-a321-0a5843143288,a099ff38-0859-11eb-a321-0a5843143288,e7c753e1-b758-11ea-a321-0a5843143288,8dbc3afc-9985-11ea-a321-0a5843143288,d478e59f-f0d0-11ea-a321-0a5843143288,60d3311c-a310-11eb-ae4a-0a5843143288,cd715e84-fe74-11eb-ae4a-0a5843143288,...,65db4a26-675d-11eb-a321-0a5843143288,d5856e06-a48d-11ea-a321-0a5843143288,7d47f6ab-1278-11eb-a321-0a5843143288,271dda07-06b4-11eb-a321-0a5843143288,b154f549-1a74-11ec-ae4a-0a5843143288,d64dae66-8bc9-11ea-a321-0a5843143288,9175ed1f-b426-11ea-a321-0a5843143288,345a9106-e1c9-11ea-a321-0a5843143288,6aec7881-3ade-11eb-a321-0a5843143288,62026c30-9a61-11eb-ae4a-0a5843143288
pm_originating_request_id,ELIG_f4408dd3-7e68-43ac-910e-b6ffd4bc3843,ELIG_475c6c60-cbda-4469-8a19-f3369a06a102,ELIG_d4fba4d6-fbf2-41eb-a6e7-95f7883daf3c,ELIG_1c9f0bbf-4dca-4d51-bd19-2e5281483f63,ELIG_6a97dda6-e8d5-48f4-80bc-bb1b11b0a598,ELIG_1a178bc6-10eb-435d-a968-be69b2c36443,ELIG_7bd759a5-6971-47b6-bce4-24a67b19f5fa,ELIG_ce780b03-91c2-44fa-ad00-2f1f66fc009a,ELIG_7f878b05-5699-4453-ac71-85a70048f1d4,ELIG_8b4e872b-bffa-4cc6-8655-2e55df639ded,...,ELIG_c1979263-f76a-44ca-b81b-75e622ebb02b,ELIG_32d5f99b-890c-4e43-af60-191c949575cc,ELIG_37cb0f61-9eb3-4e65-864e-be320eb7ee6e,ELIG_9df63485-2fae-4925-9975-5371b06e576d,ELIG_d133e87a-ef40-492b-9ce3-b9fab79b33d3,ELIG_6dd9d295-2613-4e29-adf4-d8efabbadb5c,ELIG_5a4f0a52-8068-4901-92b2-d95fe6045787,ELIG_6de20911-5432-432d-816f-6fd9f29ed440,ELIG_49289103-5d08-41b8-8558-93a1b16d1b54,ELIG_65f02f7a-5245-4c92-9369-fc2c7a518930
at_merchant_id,65655,55477,36979,37681,47350,37390,26187,45023,62815,65885,...,57149,36323,49240,46213,66940,15638,36917,44334,54947,62497
ct_originating_application_id,109678,79732,22135,24898,48145,43250,14647,40975,93015,110627,...,80303,18336,56965,46750,114249,17308,21708,36318,69849,90216
dot_record_create_date,2021-08-07 13:07:54,2021-01-08 02:33:31,2020-06-22 11:47:15,2020-06-29 12:06:17,2020-10-07 04:58:31,2020-06-26 02:58:37,2020-05-19 04:02:43,2020-09-07 06:10:44,2021-04-22 02:13:50,2021-08-16 09:31:59,...,2021-02-05 02:54:27,2020-06-02 04:59:45,2020-10-20 02:04:15,2020-10-05 02:40:52,2021-09-21 00:41:44,2020-05-01 16:36:10,2020-06-22 01:21:14,2020-08-19 03:08:31,2020-12-10 11:54:19,2021-04-11 01:01:01
dot_record_update_date,2021-08-07 13:11:06,2021-01-08 02:45:33,2020-06-22 11:52:21,2020-06-29 12:13:27,2020-10-07 05:10:12,2020-06-26 03:06:16,2020-05-19 04:08:26,2020-09-07 06:14:08,2021-04-22 02:23:40,2021-08-16 10:58:47,...,2021-02-05 02:58:40,2020-06-02 05:03:58,2020-10-20 02:10:40,2020-10-05 02:48:56,2021-09-21 00:46:09,2020-05-01 16:46:09,2020-06-22 01:29:55,2020-08-19 03:12:07,2020-12-10 12:01:31,2021-04-11 01:06:16
dot_full_name,Mindy Ng yim kun,FARAFADYLLA BT ZULKIFLY,Khoo edwin,MUHAMMAD AIDIL AMIRUL BIN ROSNIZAM,Abdullah Suhaimi,Intan nazira binti rosli,gul zaman bin gull rahman,Kaiyisah binti hamil,AZURA BINTI ALI SUPIA,Raymond David,...,Chan Mann Shih,STEPFEN YEOH LIAN KIAT,Fadhilah binti md sidek,MOHD SUHAIMI BIN KHAULI,HARITH FADZILAH BIN MAHAZIR,zulaikah intan noratika bt ibrahim,Siti nor ain rahim,Nur Azila Binti Hussin,EIDDIL EIRMIZZAD BIN AFFZAL,CHAI MEI QI
dot_identity_card,831228-07-5226,820512-11-5156,760729-07-5667,980802-56-5051,821229-03-5139,920422-08-6580,800430-11-5437,891030-08-5220,810914-08-6280,740813-14-5105,...,940511-07-5726,881209-06-5191,851029-10-5670,840312-11-5483,780822-08-6561,911225-08-6624,890107-11-5682,930721-01-6506,950306-10-6107,991118-01-5938
dot_owner_phone_number,60165570097,60137526609,60133625763,601163391001,60199753082,60189069201,60163342786,60189501090,60173387516,60105215216,...,60135181876,60169608083,60132332238,60199852755,60173769686,60127163876,60129091873,60127318457,60182050255,60123259603
...,...,...,...,...,...,...,...,